# Import Data

In [26]:
import mne
filepath = "/workspace/data/EEG/data/"

ep1 = mne.read_epochs(filepath + "epochs/A01-epo.fif", preload=True)

Reading /workspace/data/EEG/data/epochs/A01-epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     800.78 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
4200 matching events found
No baseline correction applied
0 projection items activated


In [27]:
labels = ep1.events[:, -1]

In [28]:
X = ep1.get_data() * 1000
y = labels-1

In [29]:
kernels, chans, samples = 16, 8, 257

In [30]:
X = X.reshape(X.shape[0], samples, chans)

In [31]:
# take 50/20/20 percent of the data to train/validate/test
X_train      = X[0: int(0.6*len(y)),]
Y_train      = y[0: int(0.6*len(y)),]
X_validate   = X[int(0.6*len(y)) : int(0.8*len(y)) ,]
Y_validate   = y[int(0.6*len(y)) : int(0.8*len(y)) ,]
X_test       = X[int(0.8*len(y)) : len(y) ,]
Y_test       = y[int(0.8*len(y)) : len(y) ,]


In [32]:

X_train      = X_train.reshape(X_train.shape[0], samples, chans)
X_validate   = X_validate.reshape(X_validate.shape[0], samples, chans)
X_test       = X_test.reshape(X_test.shape[0], samples, chans)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (2520, 257, 8)
2520 train samples
840 test samples


In [21]:
X.shape















(4200, 257, 8)

In [1]:
import numpy as np

CE = np.load('/workspace/data/EEG/data/epochs/A-epo.npy')
lab = np.load("/workspace/data/EEG/data/epochs/A-labels.npy")

In [2]:
rCE = CE.reshape(CE.shape[0], CE.shape[1], CE.shape[3], CE.shape[2])
tCE = np.transpose(CE,(0,1,3,2))

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8
#
# Montserrat Alvarado <amontserrat@gmail.com> / Gibran Fuentes-Pineda <gibranfp@unam.mx>
# DMAS-UAM / IIMAS-UNAM
# 2019
#
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

def cecotti_normal(shape, dtype = None, partition_info = None):
    '''
    Initializer proposed by Cecotti et al. 2011:
    https://ieeexplore.ieee.org/document/5492691
    '''
    if len(shape) == 1:
        fan_in = shape[0]
    elif len(shape) == 2:
        fan_in = shape[0]
    else:
        receptive_field_size = 1
        for dim in shape[:-2]:
            receptive_field_size *= dim
        fan_in = shape[-2] * receptive_field_size

    return K.random_normal(shape, mean = 0.0, stddev = (1.0 / fan_in))

def scaled_tanh(z):
    '''
    Scaled hyperbolic tangent activation function, as proposed
    by Lecun 1989:
    http://yann.lecun.com/exdb/publis/pdf/lecun-89.pdf

    See also Lecun et al. 1998:
    http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf
    '''
    return 1.7159 * K.tanh((2.0 / 3.0) * z)

get_custom_objects().update({'scaled_tanh': Activation(scaled_tanh)})

def CNN1(Chans = 6, Samples = 206):
    eeg_input    = Input(shape = (Samples, Chans))

    block1       = Conv1D(10, 1, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(eeg_input)
    block1       = Activation('scaled_tanh')(block1)

    block1       = Conv1D(50, 13, padding = 'same',
                          data_format = 'channels_last',
                          bias_initializer = cecotti_normal,
                          kernel_initializer = cecotti_normal,
                          use_bias = True)(block1)
    block1       = Activation('scaled_tanh')(block1)

    flatten      = Flatten(name = 'flatten')(block1)
    dense        = Dense(100, activation = 'sigmoid')(flatten)
    prediction   = Dense(2, activation = 'sigmoid')(dense)

    return Model(inputs = eeg_input, outputs = prediction, name = 'CNN1')

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8
#
# Gibran Fuentes-Pineda <gibranfp@unam.mx>
# IIMAS, UNAM
# 2018
#
"""
Script to evaluate the CNN1 architecture (Lawhern et al., 2018) for single-trial subject-dependent P300 detection
"""
import argparse
import sys
import numpy as np
#from CNN1 import CNN1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
#from tensorflow import set_random_seed
from sklearn.model_selection import *
from utils import *
import tensorflow.keras.backend as K

def evaluate_subject_models(data, labels, modelpath, subject):
    """
    Trains and evaluates CNN1 for each subject in the P300 Speller database
    using repeated stratified K-fold cross validation.
    """
    n_sub = data.shape[0]
    n_ex_sub = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]
    aucs = np.zeros(5 * 10)
    print("Training for subject {0}: ".format(subject))
    cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 123)
    for k, (t, v) in enumerate(cv.split(data[subject], labels[subject])):
        X_train, y_train, X_test, y_test = data[subject, t, :, :], labels[subject, t], data[subject, v, :, :], labels[subject, v]
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 456)
        print('Partition {0}: X_train = {1}, X_valid = {2}, X_test = {3}'.format(k, X_train.shape, X_valid.shape, X_test.shape))

        # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_valid = sc.transform(X_valid)
        X_test = sc.transform(X_test)

        model = CNN1(Chans = n_channels, Samples = n_samples)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Early stopping setting also follows EEGNet (Lawhern et al., 2018)
        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
        history = model.fit(X_train,
                            to_categorical(y_train),
                            batch_size = 256,
                            epochs = 200,
                            validation_data = (X_valid, to_categorical(y_valid)),
                            callbacks = [es])

        proba_test = model.predict(X_test)
        aucs[k] = roc_auc_score(y_test, proba_test[:, 1])
        print('S{0}, P{1} -- AUC: {2}'.format(subject, k, aucs[k]))
        K.clear_session()

    np.savetxt(modelpath + '/s' + str(subject) + '_aucs.npy', aucs)
    return aucs

In [9]:
import argparse
import sys
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
#from tensorflow import set_random_seed
from sklearn.model_selection import *
#from CNN1 import CNN1
from utils import *
import tensorflow.keras.backend as K

def evaluate_cross_subject_model(data, labels, modelpath):
    """
    Trains and evaluates CNN1 for each subject in the P300 Speller database
    using random cross validation.
    """
    n_sub = data.shape[0]
    n_ex_sub = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]

    aucs = np.zeros(n_sub)

    data = data.reshape((n_sub * n_ex_sub, n_samples, n_channels))
    labels = labels.reshape((n_sub * n_ex_sub))
    groups = np.array([i for i in range(n_sub) for j in range(n_ex_sub)])

    cv = LeaveOneGroupOut()
    for k, (t, v) in enumerate(cv.split(data, labels, groups)):
        X_train, y_train, X_test, y_test = data[t], labels[t], data[v], labels[v]

        rg = np.random.choice(t, 1)
        sv = groups[t] == groups[rg]
        st = np.logical_not(sv)
        X_train, y_train, X_valid, y_valid = data[t][st], labels[t][st], data[t][sv], labels[t][sv]
        print("Partition {0}: train = {1}, valid = {2}, test = {3}".format(k, X_train.shape, X_valid.shape, X_test.shape))
        print("Groups train = {0}, valid = {1}, test = {2}".format(np.unique(groups[t][st]),
                                                                   np.unique(groups[t][sv]),
                                                                   np.unique(groups[v])))

        # channel-wise feature standarization
        sc = EEGChannelScaler(n_channels = n_channels)
        X_train = sc.fit_transform(X_train)
        X_valid = sc.transform(X_valid)
        X_test = sc.transform(X_test)

        model = CNN1(Chans = n_channels, Samples = n_samples)
        print(model.summary())
        model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

        es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50, restore_best_weights = True)
        model.fit(X_train,
                  to_categorical(y_train),
                  batch_size = 256,
                  epochs = 200,
                  validation_data = (X_valid, to_categorical(y_valid)),
                  callbacks = [es])

        proba_test = model.predict(X_test)
        aucs[k] = roc_auc_score(y_test, proba_test[:, 1])
        print('P{0} -- AUC: {1}'.format(k, aucs[k]))
        K.clear_session()

    np.savetxt(modelpath + '/aucs.npy', aucs)
    return aucs

In [10]:
l = lab-1

In [17]:
auc_tsub = evaluate_subject_models(tCE, l, "/workspace/data/EEG/models/", 0)

Training for subject 0: 
Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d (Conv1D)             (None, 257, 10)           90        
                                                                 
 activation (Activation)     (None, 257, 10)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 50)           6550      
                                                                 
 activation_1 (Activation)   (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
                

In [21]:
np.mean(auc_tsub)

0.9338746938775511

In [19]:
auc_resub = evaluate_subject_models(rCE, l, "/workspace/data/EEG/models/", 0)

Training for subject 0: 
Partition 0: X_train = (2688, 257, 8), X_valid = (672, 257, 8), X_test = (840, 257, 8)
Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d (Conv1D)             (None, 257, 10)           90        
                                                                 
 activation (Activation)     (None, 257, 10)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 50)           6550      
                                                                 
 activation_1 (Activation)   (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
                

In [22]:
np.mean(auc_resub)

0.9317146938775511

In [11]:
auc_cross_tsub = evaluate_cross_subject_model(tCE, l, "/workspace/data/EEG/models/")


Partition 0: train = (25200, 257, 8), valid = (4200, 257, 8), test = (4200, 257, 8)
Groups train = [1 2 3 4 6 7], valid = [5], test = [0]


2022-03-18 00:29:39.168530: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "CNN1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 257, 8)]          0         
                                                                 
 conv1d (Conv1D)             (None, 257, 10)           90        
                                                                 
 activation_1 (Activation)   (None, 257, 10)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 257, 50)           6550      
                                                                 
 activation_2 (Activation)   (None, 257, 50)           0         
                                                                 
 flatten (Flatten)           (None, 12850)             0         
                                                                 
 dense (Dense)               (None, 100)               1285100

2022-03-18 00:29:39.930405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:bc:00.0, compute capability: 7.0


None
Epoch 1/200


2022-03-18 00:29:41.957131: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-03-18 00:29:42.414866: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


99/99 [==============================] - 3s 9ms/step - loss: 0.4010 - val_loss: 0.3548
Epoch 2/200
99/99 [==============================] - 0s 5ms/step - loss: 0.3736 - val_loss: 0.3635

In [12]:
np.mean(auc_cross_tsub)


0.7843588265306122